In [ ]:
%pip install --upgrade pandas
%pip install --upgrade numpy
%pip install --upgrade matplotlib
%pip install --upgrade seaborn
%pip install --upgrade scikit-learn-intelex
%pip install --upgrade dpcpp-cpp-rt
%pip install --upgrade xgboost
%pip install --upgrade sklearn-ex

In [ ]:
from sklearnex import patch_sklearn
patch_sklearn()

Importing Libraries

In [ ]:
df = pd.read_csv('all.csv')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.size

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df['names'].unique()

In [ ]:
df.dtypes

In [ ]:
df['names'].value_counts()

In [ ]:
import seaborn as sns

sns.heatmap(df.corr(),annot=True)